In [1]:
(1024 * 64) / (32*1024)

2.0

In [ ]:
import sys
sys.path.append("../src")

import wandb
import pretrain.settings as s
from pretrain.utils import ModelSummary
from pretrain.models import GPT
from pathlib import Path
from pretrain.utils import ModelCheckpointManager
from dataset.dataset_classes import UltraChat200kDataLoaderLite
from peft import LoraConfig, get_peft_model, TaskType

we need CUDA for DDP so falling back to CPU


In [2]:
model = GPT()
ModelSummary.summary(model)

Model size: 475.03 MB
Trainable parameters: 124.53M
Non-trainable parameters: 0


In [3]:
run = wandb.init(dir=s.logs_root_path)
artifact = run.use_artifact('sampath017/GPT3-124M/model_checkpoint_train_step_17000_val_loss_3.08:v0', type='model')
artifact_dir = artifact.download(s.models_root_path)
artifact_dir

wandb: Currently logged in as: sampath017 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model_checkpoint_train_step_17000_val_loss_3.08:v0, 1425.29MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.1 (453.5MB/s)


'C:\\Users\\sampath\\Dev\\GPT\\models'

In [4]:
checkpoint_files = list(Path(artifact_dir).glob("*.pt"))
if not checkpoint_files:
    raise FileNotFoundError(f"No .pt files found in {artifact_dir}")

checkpoint_path = checkpoint_files[0]
print("Using checkpoint:", checkpoint_path)

# Load model + optimizer
model, _ = ModelCheckpointManager.load_checkpoint(checkpoint_path, model)
model

Using checkpoint: C:\Users\sampath\Dev\GPT\models\model_checkpoint_train_step_17000_val_loss_3.08.pt


GPT(
  (transformer): ModuleDict(
    (token_embedding_table): Embedding(50304, 768)
    (position_embedding_table): Embedding(1024, 768)
    (blocks): Sequential(
      (0): Block(
        (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (self_attention): CausalSelfAttention(
          (attn): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (feed_forward): FeedForward(
          (ff): Sequential(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=3072, out_features=768, bias=True)
            (3): Dropout(p=0.2, inplace=False)
          )
        )
      )
      (1): Block(
        (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (self_attention): CausalSelfAttention(
          (attn): Li

In [5]:
# generations = generate(model)

In [6]:
dataloader = UltraChat200kDataLoaderLite(split="val")
dataloader

found 1 shards for split val


In [7]:
x, y = dataloader.next_batch()
x.shape, y.shape

(torch.Size([64, 1024]), torch.Size([64, 1024]))

In [8]:
lora_config = LoraConfig(
    r=16,              # rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["attn", "proj"]  # GPT-2 attention layers
)

# wrap model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 884,736 || all params: 125,410,944 || trainable%: 0.7055


In [9]:
from torch import optim
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(2):
    for input_ids, labels in dataloader:
        logits, loss = model(input_ids, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} | loss {loss.item():.4f}")


TypeError: 'UltraChat200kDataLoaderLite' object is not iterable